## Business Understanding
The loan providing companies find it hard to give loans to the people due to their insufficient or non-existent credit history. Because of that, some consumers use it to their advantage by becoming a defaulter. Suppose you work for a consumer finance company which specialises in lending various types of loans to urban customers. You have to use EDA to analyse the patterns present in the data. This will ensure that the applicants capable of repaying the loan are not rejected.

 

When the company receives a loan application, the company has to decide for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company.

 

The data given below contains the information about the loan application at the time of applying for the loan. It contains two types 
The client with payment difficulties: he/she had late payment more than X days on at least one of the first Y instalments of the loan in our sample,

All other cases: All other cases when the payment is paid on time.

 

When a client applies for a loan, there are four types of decisions that could be taken by the client/company):

Approved: The Company has approved loan Application

Cancelled: The client cancelled the application sometime during approval. Either the client changed her/his mind about the loan or in some cases due to a higher risk of the client, he received worse pricing which he did not want.

Refused: The company had rejected the loan (because the client does not meet their requirements etc.).

Unused offer:  Loan has been cancelled by the client but at different stages of the process.

In this case study, you will use EDA to understand how consumer attributes and loan attributes influence the tendency to default.ency to default.
ency to default.of scenarios:

# **Business Objectives**
This case study aims to identify patterns which indicate if a client has difficulty paying their instalments which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc. This will ensure that the consumers capable of repaying the loan are not rejected. Identification of such applicants using EDA is the aim of this case study.

 

In other words, the company wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment.

To develop your understanding of the domain, you are advised to independently research a little about risk analytics - understanding the types of variables and their significance should be enough.

### Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')  #importings warnings since most seaborn functions are deprecated and to make notebook look clean

### Application Dataset:
- Dataset consist of columns thats about personal details of client.
- Key Column which is **TARGET** rates risk about client.
- Some columns about document collected
- Some columns about number of Enquiries from client on certain time frame , data collected from Central Bureau.

## 1 : DATA CLEANING

#### Methods Taken in Data Cleaning:
- Removing columns with more than 40% nulls.
- Finding and Removing outliers.
- Imputing Mean,Median based Distribution of data [Median for Skewed Distribution,Mean for Normal Distribution]
- Modifying the columns for better insights [eg: Nulls in Occupation type has been changed into Retired who has Income Type Pensioner]
- Feature Engineering Few Columns.
- Standardised Categorical Values of Few Columns.
- Kept abnormal values in some columns as imputed nulls created artificial outliers.<br>
`Note: Categorizing by Numerical and Categorical data is done in end after cleaning Nulls`

In [ ]:
app_df = pd.read_csv('application_data.csv')

In [ ]:
app_df.shape

In [ ]:
app_df.dtypes

In [ ]:
# Majority of Columns are truncated in middle and not visible in cell,lets fix it using set_options
pd.set_option('display.max_columns',None) # for columns 
pd.set_option('display.max_rows',None) # for rows

In [ ]:
app_df.dtypes

In [ ]:
app_df.head(10)

In [ ]:
app_df.info()

### Application Dataset has 
- 65 float64 columns
- 41 int64  columns
- 16 object columns

In [ ]:
# lets create functions to check null percentage in dataframe.
def checknull(n):
    return app.isnull().sum().sort_values(ascending=False) / n.shape[0] * 100

In [ ]:
def fillmedian(df, columns):
    for column in columns:
        median = df[column].median()
        df[column].fillna(median, inplace=True)

In [ ]:
# calling created function to check null percentage
checknull(app_df)

#### Dropping Columns with more than 40% Nulls

In [ ]:
null_percentage = checknull(app_df)
highnull_percentage = null_percentage[null_percentage > 40].index
highnull_percentage.shape  # checking how many columns with nulls more than 40%

In [ ]:
app_df.drop(columns=highnull_percentage,axis=1, inplace=True)

In [ ]:
app_df.shape  # as we can see 49 columns has been dropped

#### OCCUPATION_TYPE Column Cleaning



In [ ]:
app_df.OCCUPATION_TYPE.value_counts()

In [ ]:
app_df.OCCUPATION_TYPE.describe()

In [ ]:
app_df.OCCUPATION_TYPE.isnull().sum()

##### Will check parallel column of "NAME_INCOME_TYPE" and reduce the nulls in occupation type.

In [ ]:
null_occupation = app_df[app_df['OCCUPATION_TYPE'].isnull()]

In [ ]:
null_occupation.groupby("NAME_INCOME_TYPE",dropna=False)["SK_ID_CURR"].count()

In [ ]:
app_df.OCCUPATION_TYPE.unique()

In [ ]:
app_df.loc[app_df["NAME_INCOME_TYPE"] == "Pensioner", "OCCUPATION_TYPE"] = "Retired"

In [ ]:
app_df.OCCUPATION_TYPE.mode()[0]

#Now we can fill the nulls with mode value

In [ ]:
app_df.OCCUPATION_TYPE.fillna(app_df.OCCUPATION_TYPE.mode()[0],inplace=True)

#### EXT_SOURCE_3 Column Cleaning

In [ ]:
app_df.EXT_SOURCE_3.describe()

In [ ]:
app_df.EXT_SOURCE_3.plot.box()

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot(app_df['EXT_SOURCE_3'],kde=True)
plt.title('Density Plot of EXT_SOURCE_3')
plt.xlabel('EXT_SOURCE_3')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

In [ ]:
# since this column is left skewed, so going with median imputation

In [ ]:
app_df.EXT_SOURCE_3.skew()

In [ ]:
app_df.EXT_SOURCE_3.fillna(app_df.EXT_SOURCE_3.median(),inplace = True)

In [ ]:
sns.histplot(data=app_df,x='EXT_SOURCE_3',bins=50)

In [ ]:
sns.boxplot(data=app_df,y='EXT_SOURCE_3')

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot(app_df['EXT_SOURCE_3'],kde=True)
plt.title('Density Plot of EXT_SOURCE_3')
plt.xlabel('EXT_SOURCE_3')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

------

#### Credit Bureau Columns Cleaning

In [ ]:
cred_b_columns = ["AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK","AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"]
cred_b_columns

In [ ]:
app_df.loc[:, cred_b_columns].describe()

Lets find out outlier for each column using IQR , since data is skewed.

In [ ]:
IQR = 3-0
lower_bound = 0 - 1.5 * IQR
upper_bound = 3 + 1.5 * IQR

print("Lower Bound of Credit Bureau Year : ", lower_bound)
print("Upper Bound of Credit Bureau Year : ", upper_bound)

In [ ]:
app_df = app_df[~(app_df.AMT_REQ_CREDIT_BUREAU_YEAR > 8)]

In [ ]:
app_df.AMT_REQ_CREDIT_BUREAU_QRT.unique()

In [ ]:
app_df = app_df[~(app_df.AMT_REQ_CREDIT_BUREAU_QRT > 8)]

In [ ]:
app_df = app_df[~(app_df.AMT_REQ_CREDIT_BUREAU_MON > 8)]

In [ ]:
app_df = app_df[~(app_df.AMT_REQ_CREDIT_BUREAU_WEEK > 8)]

In [ ]:
app_df = app_df[~(app_df.AMT_REQ_CREDIT_BUREAU_DAY > 8)]

In [ ]:
app_df[cred_b_columns] = app_df[cred_b_columns].fillna(app_df[cred_b_columns].median() )

In [ ]:
checknull(app_df[cred_b_columns])

#### Feature Engineering On Credit Bureau Columns to Total Enquiry Per Year by Client

In [ ]:
app_df["Total_Req_Credit_Enq_Year"] = app_df[cred_b_columns].sum(axis=1)

In [ ]:
app_df.Total_Req_Credit_Enq_Year.describe()

In [ ]:
app_df.Total_Req_Credit_Enq_Year.plot.box()

In [ ]:
app_df.Total_Req_Credit_Enq_Year.plot.hist(bins=15)

#### NAME_TYPE_SUITE Column Cleaning

In [ ]:
app_df.NAME_TYPE_SUITE.value_counts()

In [ ]:
app_df.NAME_TYPE_SUITE.isnull().sum()

In [ ]:
app_df.NAME_TYPE_SUITE.mode()[0]

In [ ]:
app_df.NAME_TYPE_SUITE.fillna(app_df.NAME_TYPE_SUITE.mode()[0],inplace = True)

#### creating new Column which has 2 Category, Accompanied ,Unaccompanied

In [ ]:
app_df['ACCOMPANIED_STATUS'] = app_df['NAME_TYPE_SUITE'].apply(
    lambda x: 'Accompanied' if x in ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people'] else 'Unaccompanied'
)


In [ ]:
app_df['ACCOMPANIED_STATUS'].value_counts()

In [ ]:
app_df['NAME_TYPE_SUITE'] = app_df['NAME_TYPE_SUITE'].apply(lambda x: 'Accompanied' if x in ['Other_B', 'Other_A', 'Group of people'] else 'Others')

#### Social Circle Columns Cleaning

In [ ]:
checknull(app_df).head(4)

In [ ]:
social_circle = checknull(app_df).head(4).index

In [ ]:
app_df[social_circle].describe()

In [ ]:
def remove_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data >= lower_bound) & (data <= upper_bound)]

In [ ]:
app_df['OBS_60_CNT_SOCIAL_CIRCLE'] = remove_outliers(app_df['OBS_60_CNT_SOCIAL_CIRCLE'])
app_df['DEF_60_CNT_SOCIAL_CIRCLE'] = remove_outliers(app_df['DEF_60_CNT_SOCIAL_CIRCLE'])
app_df['OBS_30_CNT_SOCIAL_CIRCLE'] = remove_outliers(app_df['OBS_30_CNT_SOCIAL_CIRCLE'])
app_df['DEF_30_CNT_SOCIAL_CIRCLE'] = remove_outliers(app_df['DEF_30_CNT_SOCIAL_CIRCLE'])

In [ ]:
app_df[social_circle].describe()

In [ ]:
app_df['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(app_df['OBS_60_CNT_SOCIAL_CIRCLE'].median(), inplace=True)
app_df['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(app_df['DEF_60_CNT_SOCIAL_CIRCLE'].median(), inplace=True)
app_df['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(app_df['OBS_30_CNT_SOCIAL_CIRCLE'].median(), inplace=True)
app_df['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(app_df['DEF_30_CNT_SOCIAL_CIRCLE'].median(), inplace=True)

In [ ]:
app_df['DEF_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

In [ ]:
print(app_df['DEF_30_CNT_SOCIAL_CIRCLE'].dtype)
print(app_df['DEF_30_CNT_SOCIAL_CIRCLE'].unique())

In [ ]:
print(app_df['DEF_60_CNT_SOCIAL_CIRCLE'].dtype)
print(app_df['DEF_60_CNT_SOCIAL_CIRCLE'].unique())

#### EXT_SOURCE_2 Column Cleaning

In [ ]:
app_df.EXT_SOURCE_2.describe()

In [ ]:
app_df.EXT_SOURCE_2.plot.box()
plt.show()

In [ ]:
app_df.EXT_SOURCE_2.plot.hist()
plt.show()

In [ ]:
app_df.EXT_SOURCE_2.median()

In [ ]:
app_df.EXT_SOURCE_2.fillna(app_df.EXT_SOURCE_2.median(),inplace=True)

#### AMT_GOODS_PRICE Column Cleaning

In [ ]:
app_df.AMT_GOODS_PRICE.plot.box()
plt.show()

In [ ]:
app_df.AMT_GOODS_PRICE.plot.hist()
plt.show()

In [ ]:
app_df.AMT_GOODS_PRICE.median()

In [ ]:
app_df.AMT_GOODS_PRICE.describe()

In [ ]:
app_df.AMT_GOODS_PRICE.fillna(app_df.AMT_GOODS_PRICE.median(),inplace=True)

#### AMT_ANNUITY Column cleaning

In [ ]:
app_df.AMT_ANNUITY.describe()

In [ ]:
app_df.AMT_ANNUITY.plot.box()

In [ ]:
app_df.AMT_ANNUITY.plot.hist()

In [ ]:
app_df[app_df.AMT_ANNUITY > 200000]

In [ ]:
app_df.AMT_ANNUITY.median()

In [ ]:
app_df.AMT_ANNUITY.fillna(app_df.AMT_ANNUITY.median(),inplace=True)

#### CNT_FAM_MEMBERS Column Cleaning

In [ ]:
app_df.CNT_FAM_MEMBERS.describe()

In [ ]:
app_df.CNT_FAM_MEMBERS.plot.hist()

In [ ]:
app_df.CNT_FAM_MEMBERS.plot.box()

In [ ]:
app_df.NAME_FAMILY_STATUS.unique()

In [ ]:
app_df.NAME_HOUSING_TYPE.unique()

In [ ]:
app_df.NAME_TYPE_SUITE.unique()

In [ ]:
app_df[app_df['CNT_FAM_MEMBERS'] > 7 ].shape

In [ ]:
app_df = app_df[~((app_df['CNT_FAM_MEMBERS'] > 7 ) &  (app_df['NAME_FAMILY_STATUS'] == 'Single / not married'))]

`Removed only rows who has family size of more than 7 and single since its impossible`

In [ ]:
app_df[app_df['CNT_FAM_MEMBERS'] >7 ]

`Even though size of this is small, still i feel like its better to keep these rows and check if family size actually affects client being defaulters` <br> `"this is hypothesis of mine" so im keeping this rows to check later in analysis steps.`

In [ ]:
app_df.CNT_FAM_MEMBERS.median()

In [ ]:
app_df.CNT_FAM_MEMBERS.fillna(app_df.CNT_FAM_MEMBERS.median(),inplace=True)

#### DAYS_LAST_PHONE_CHANGE Column cleaning

In [ ]:
app_df.DAYS_LAST_PHONE_CHANGE.describe()

In [ ]:
app_df.DAYS_LAST_PHONE_CHANGE.median()

In [ ]:
app_df.DAYS_LAST_PHONE_CHANGE.plot.hist()

In [ ]:
app_df.DAYS_LAST_PHONE_CHANGE.fillna(app_df.DAYS_LAST_PHONE_CHANGE.median(),inplace=True)

#### Creating new bin columns for columns below : 
- AMT_INCOME_TOTAL,
- AMT_CREDIT,
- AMT_ANNUITY,
- AMT_GOODS_PRICE,
- DAYS_BIRTH,
- DAYS_EMPLOYED,
- DAYS_REGISTRATION,
- DAYS_ID_PUBLISH 

In [ ]:
bins = [0, 300000, 600000, 1000000, 1500000, 2000000, float('inf')]
labels = ['0-3 lakhs', '3-6 lakhs', '6-10 lakhs', '10-15 lakhs', '15-20 lakhs', '20+ lakhs']
app_df['Income_Category'] = pd.cut(app_df['AMT_INCOME_TOTAL'], bins=bins, labels=labels, right=False)
print(app_df[['AMT_INCOME_TOTAL', 'Income_Category']].head(10))

In [ ]:
credit_bins = [0, 100000, 300000, 500000, 750000, 1000000, 2000000, float('inf')]
credit_labels = ['0-1 lakh', '1-3 lakhs', '3-5 lakhs', '5-7.5 lakhs', '7.5-10 lakhs', '10-20 lakhs', '20+ lakhs']
app_df['Credit_Category'] = pd.cut(app_df['AMT_CREDIT'], bins=credit_bins, labels=credit_labels, right=False)
print(app_df[['AMT_CREDIT', 'Credit_Category']].head())

In [ ]:
annuity_bins = [0, 10000, 20000, 30000, 40000, 50000, 70000, float('inf')]
annuity_labels = ['0-10k', '10k-20k', '20k-30k', '30k-40k', '40k-50k', '50k-70k', '70k+']
app_df['Annuity_Category'] = pd.cut(app_df['AMT_ANNUITY'], bins=annuity_bins, labels=annuity_labels, right=False)
print(app_df[['AMT_ANNUITY', 'Annuity_Category']].head())

In [ ]:
goods_price_bins = [0, 100000, 300000, 500000, 750000, 1000000, 2000000, float('inf')]
goods_price_labels = ['0-1 lakh', '1-3 lakhs', '3-5 lakhs', '5-7.5 lakhs', '7.5-10 lakhs', '10-20 lakhs', '20+ lakhs']
app_df['Goods_Price_Category'] = pd.cut(app_df['AMT_GOODS_PRICE'], bins=goods_price_bins, labels=goods_price_labels, right=False)
print(app_df[['AMT_GOODS_PRICE', 'Goods_Price_Category']].head())

In [ ]:
app_df['AGE_YEARS'] = (-app_df['DAYS_BIRTH'] / 365).astype(int)
age_bins = [0, 25, 35, 45, 55, 65, float('inf')]
age_labels = ['0-25', '26-35', '36-45', '46-55', '56-65', '65+']
app_df['Age_Category'] = pd.cut(app_df['AGE_YEARS'], bins=age_bins, labels=age_labels, right=False)
print(app_df[['DAYS_BIRTH', 'AGE_YEARS', 'Age_Category']].head())

In [ ]:
app_df['EMPLOYMENT_YEARS'] = (-app_df['DAYS_EMPLOYED'] / 365).astype(int)
employment_bins = [0, 1, 3, 5, 10, 20, float('inf')]
employment_labels = ['0-1 year', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years']
app_df['Employment_Category'] = pd.cut(app_df['EMPLOYMENT_YEARS'], bins=employment_bins, labels=employment_labels, right=False)
print(app_df[['DAYS_EMPLOYED', 'EMPLOYMENT_YEARS', 'Employment_Category']].head())

In [ ]:
app_df['REGISTRATION_YEARS'] = (-app_df['DAYS_REGISTRATION'] / 365).astype(int)
registration_bins = [0, 1, 3, 5, 10, 20, float('inf')]
registration_labels = ['0-1 year', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years']
app_df['Registration_Category'] = pd.cut(app_df['REGISTRATION_YEARS'], bins=registration_bins, labels=registration_labels, right=False)
print(app_df[['DAYS_REGISTRATION', 'REGISTRATION_YEARS', 'Registration_Category']].head())

In [ ]:
app_df['ID_PUBLISH_YEARS'] = (-app_df['DAYS_ID_PUBLISH'] / 365).astype(int)
id_publish_bins = [0, 1, 3, 5, 10, 20, float('inf')]
id_publish_labels = ['0-1 year', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years']
app_df['ID_Publish_Category'] = pd.cut(app_df['ID_PUBLISH_YEARS'], bins=id_publish_bins, labels=id_publish_labels, right=False)
print(app_df[['DAYS_ID_PUBLISH', 'ID_PUBLISH_YEARS', 'ID_Publish_Category']].head())

In [ ]:
categorical_columns = list(app_df.select_dtypes(include="object").dtypes.index)

In [ ]:
categorical_columns

In [ ]:
app_df[categorical_columns[0]].unique()

In [ ]:
app_df[categorical_columns[1]].unique()

#### Cleaning Gender Column

In [ ]:
app_df[categorical_columns[1]].value_counts()

In [ ]:
app_df = app_df[~ (app_df['CODE_GENDER'] == 'XNA')]

In [ ]:
app_df['CODE_GENDER'] = app_df['CODE_GENDER'].apply(lambda x:"Male" if x == "M" else "Female")

In [ ]:
app_df[categorical_columns[1]].value_counts()

In [ ]:
app_df[categorical_columns[2]].value_counts()

In [ ]:
app_df[categorical_columns[3]].value_counts()

In [ ]:
app_df[categorical_columns[4]].value_counts()

In [ ]:
app_df[categorical_columns[5]].value_counts()

#### Cleaning NAME_INCOME_TYPE Column

In [ ]:
app_df['NAME_INCOME_TYPE'] = app_df['NAME_INCOME_TYPE'].replace({
    'Unemployed': 'Other',
    'Student': 'Other',
    'Maternity leave': 'Other'})

In [ ]:
app_df[categorical_columns[5]].value_counts()

#### NAME_EDUCATION_TYPE Column standardizing

In [ ]:
app_df[categorical_columns[6]].value_counts()

In [ ]:
app_df['NAME_EDUCATION_TYPE'] = app_df['NAME_EDUCATION_TYPE'].replace({
    'Incomplete higher': 'Other',
    'Lower secondary': 'Other',
    'Academic degree': 'Other',
    'Secondary / secondary special': 'Secondary education'})

In [ ]:
app_df[categorical_columns[6]].value_counts()

#### NAME_FAMILY_STATUS Column Cleaning

In [ ]:
app_df[categorical_columns[7]].value_counts()

In [ ]:
app_df = app_df[app_df['NAME_FAMILY_STATUS'] != 'Unknown']

#### NAME_HOUSING_TYPE Columns Cleaning

In [ ]:
app_df[categorical_columns[8]].value_counts()

In [ ]:
app_df['NAME_HOUSING_TYPE'] = app_df['NAME_HOUSING_TYPE'].replace({
    'Office apartment': 'Other',
    'Co-op apartment': 'Other'
})

In [ ]:
app_df[categorical_columns[8]].value_counts()

In [ ]:
app_df[categorical_columns[9]].value_counts()

In [ ]:
app_df[categorical_columns[10]].value_counts()

#### Cleaning ORGANIZATION_TYPE Column

In [ ]:
app_df[categorical_columns[11]].value_counts()

In [ ]:
def categorize_org(input):
    if input in ['Business Entity Type 1', 'Business Entity Type 2', 'Business Entity Type 3']:
        return 'Business Entity'
    elif input in ['Industry: type 1', 'Industry: type 2', 'Industry: type 3', 'Industry: type 4', 'Industry: type 5', 'Industry: type 6', 'Industry: type 7', 'Industry: type 8', 'Industry: type 9', 'Industry: type 10', 'Industry: type 11', 'Industry: type 12', 'Industry: type 13']:
        return 'Industry'
    elif input in ['Transport: type 1', 'Transport: type 2', 'Transport: type 3', 'Transport: type 4']:
        return 'Transport'
    elif input in ['Trade: type 1', 'Trade: type 2', 'Trade: type 3', 'Trade: type 4', 'Trade: type 5', 'Trade: type 6', 'Trade: type 7']:
        return 'Trade'
    else:
        return input

In [ ]:
app_df['ORGANIZATION_TYPE'] = app_df['ORGANIZATION_TYPE'].apply(categorize_org)


In [ ]:
app_df[categorical_columns[11]].value_counts()

In [ ]:
app_df[categorical_columns[12]].value_counts()

In [ ]:
#### Dropping some unwanted Columns before analysis step.

In [ ]:
app_df = app_df.drop(columns=['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
       'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'])

# DATA ANALYSIS

`Data imbalance check on column TARGET`

In [ ]:
app_df['TARGET'].value_counts().values

In [ ]:
Data_Imbalance = np.round(app_df['TARGET'].value_counts().values[0] /app_df['TARGET'].value_counts().values[1],2)
print(Data_Imbalance)

In [ ]:
target_counts = app_df['TARGET'].value_counts()
labels = target_counts.index
sizes = target_counts.values
explode = [0, 0.1]
plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, explode=explode, autopct='%.0f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
plt.title('Distribution of TARGET')
plt.axis('equal')
plt.show()

### This Imbalance states 3 things:
- Due to the significantly higher number of non-defaulter records compared to defaulters, there is a risk that insights derived from this dataset may be biased towards non-defaulters. This imbalance might cause to overlook critical patterns and characteristics unique to defaulters.
- If this dataset represents the entire bank's customer base, it indicates a strong prevalence of non-defaulters over defaulters. This insight suggests that the majority of the bank's customers are managing their repayments effectively. This also implies that our current methods and processes may be effective for the majority of our clients.
- The data reveals that, on average, 1 out of every 12 customers is a defaulter, translating to a defaulter rate of approximately 8.33%. This ratio is crucial for understanding the bank's risk exposure and can inform strategies for risk management and customer support.

In [ ]:
sns.set(style='whitegrid')

In [ ]:
numerical_columns = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_YEAR', 'Total_Req_Credit_Enq_Year'
]

In [ ]:
categorical_columns = [
    'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE', 'ACCOMPANIED_STATUS', 'Income_Category',
    'Credit_Category', 'Annuity_Category', 'Goods_Price_Category',
    'Age_Category', 'Employment_Category', 'Registration_Category',
    'ID_Publish_Category'
]

# Univariate Analysis on Numerical Columns

In [ ]:
for col in numerical_columns:
    # Distribution plot
    plt.figure(figsize=(12, 6))
    sns.histplot(app_df[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

# Univariate Analysis on Categorical Columns

In [ ]:
for col in categorical_columns:
    # Frequency distribution plot
    plt.figure(figsize=(12, 6))
    sns.countplot(data=app_df, x=col, order=app_df[col].value_counts().index)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.grid(True)
    plt.xticks(rotation=45,ha='right')
    plt.show()

# Bivariate Analysis on Numerical Columns

- 0.6 to 1: Strong positive correlation.
- 0.3 to 0.6: Moderate positive correlation.
- 0 to 0.3:  Weak positive correlation.
- 0 to -0.3: Weak negative correlation.
- -0.3 to -0.6: Weak negative correlation.
- -0.6 to -1: Moderate to strong negative correlation.

In [ ]:
numerical_columns = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE','DAYS_BIRTH', 'DAYS_EMPLOYED',
    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH','Total_Req_Credit_Enq_Year'
]
# removed few unwanted columns before bivariate analysis

In [ ]:
palette = sns.color_palette("rocket")

# Numerical vs Numerical without Target
for i in range(len(numerical_columns)):
    for j in range(i+1, len(numerical_columns)):
        col1 = numerical_columns[i]
        col2 = numerical_columns[j]
        
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=app_df, x=col1, y=col2, color=palette[0])
        plt.title(f'Scatter Plot of {col1} vs {col2}', fontsize=15)
        plt.xlabel(col1, fontsize=12)
        plt.ylabel(col2, fontsize=12)
        plt.show()
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(app_df[[col1, col2]].corr(), annot=True, cmap="rocket", fmt=".2f", linewidths=0.5)
        plt.title(f'Correlation Heatmap between {col1} and {col2}', fontsize=15)
        plt.show()

In [ ]:
numerical_columns = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE','DAYS_BIRTH', 'DAYS_EMPLOYED',
    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH','Total_Req_Credit_Enq_Year'
]

In [ ]:
correlation_data = {'Column': [], 'Correlation with TARGET': []}
for col in numerical_columns:
    correlation = app_df[[col, 'TARGET']].corr().iloc[0, 1]
    correlation_data['Column'].append(col)
    correlation_data['Correlation with TARGET'].append(correlation)
correlation_data

# Bivariate Analysis On Numerical Columns with Target

In [ ]:

correlation_data = {'Column': [], 'Correlation with TARGET': []}
for col in numerical_columns:
    correlation = app_df[[col, 'TARGET']].corr().iloc[0, 1]
    correlation_data['Column'].append(col)
    correlation_data['Correlation with TARGET'].append(correlation)
    
correlation_df = pd.DataFrame(correlation_data).set_index('Column')

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_df, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation of Numerical Columns with TARGET', fontsize=15)
plt.show()

In [ ]:
categorical_columns = [
    'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE', 'ACCOMPANIED_STATUS', 'Income_Category',
    'Credit_Category', 'Annuity_Category', 'Goods_Price_Category',
    'Age_Category', 'Employment_Category', 'Registration_Category',
    'ID_Publish_Category'
]

# Bivariate Analysis on Categorical Columns

In [ ]:
for i in range(len(categorical_columns)):
    for j in range(i + 1, len(categorical_columns)):
        col1 = categorical_columns[i]
        col2 = categorical_columns[j]
        
        plt.figure(figsize=(10, 6))
        cross_tab = pd.crosstab(app_df[col1], app_df[col2])
        sns.heatmap(cross_tab, annot=True, cmap='rocket', fmt='d', linewidths=0.5)
        plt.title(f'Heatmap of Cross-Tabulation between {col1} and {col2}', fontsize=15)
        plt.xlabel(col2, fontsize=12)
        plt.ylabel(col1, fontsize=12)
        plt.show()


# Bivariate Analysis on Categorical Columns with Target

In [ ]:
palette = {0: 'lightgreen', 1: 'red'}
for col in categorical_columns:
    plt.figure(figsize=(12, 6))
    sns.countplot(data=app_df, x=col, hue='TARGET', palette=palette)
    plt.title(f'Count Plot of {col} vs TARGET', fontsize=15)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='TARGET', title_fontsize='13', fontsize='11')
    
    plt.xticks(rotation=45,ha='right')
    plt.tight_layout()
    plt.show()

# DATA CLEANING ON PREVIOUS_APPLICATION DATASET

In [ ]:
prev_appdf = pd.read_csv('previous_application.csv')

In [ ]:
prev_appdf.head()

In [ ]:
prev_appdf.shape

In [ ]:
nullspercentage = checknull(prev_appdf)
highnulls = nullspercentage[nullspercentage > 40].index
highnulls.shape

In [ ]:
prev_appdf.dtypes

In [ ]:
prev_appdf.drop(columns=highnulls,axis=1,inplace=True)

In [ ]:
fillmedian(prev_appdf,['CNT_PAYMENT','AMT_GOODS_PRICE','AMT_ANNUITY','CNT_PAYMENT'])

In [ ]:
prev_appdf.PRODUCT_COMBINATION.value_counts()

In [ ]:
prev_appdf['PRODUCT_COMBINATION'].fillna('Unknown', inplace=True)

In [ ]:
prev_appdf['PRODUCT_COMBINATION'] = prev_appdf['PRODUCT_COMBINATION'].str.strip().str.lower().head()

In [ ]:
prev_appdf.columns

In [ ]:
num_cols =  ['AMT_ANNUITY',"AMT_APPLICATION","AMT_CREDIT","AMT_GOODS_PRICE","DAYS_DECISION"]

In [ ]:
prev_appdf[num_cols].head()

In [ ]:
cat_cols = [
    'NAME_CONTRACT_TYPE',
    'NAME_CASH_LOAN_PURPOSE',
    'NAME_CONTRACT_STATUS',
    'NAME_PAYMENT_TYPE',
    'CODE_REJECT_REASON',
    'NAME_CLIENT_TYPE',
    'NAME_GOODS_CATEGORY',
    'CHANNEL_TYPE',
    'NAME_SELLER_INDUSTRY',
    'PRODUCT_COMBINATION'
]

In [ ]:
prev_appdf[cat_cols].head()

#### Univariate Analysis on Numerical Columns

In [ ]:
for col in num_cols:
    plt.figure(figsize=(10, 6))
    sns.histplot(prev_appdf[col], kde=True, color='skyblue')
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

#### BIVARIATE ANALYSIS on Numerical Columns

In [ ]:
sns.pairplot(prev_appdf[num_cols])
plt.suptitle('Pairplot of Numerical Variables', y=1.02)
plt.show()

#### Univariate Analysis on Categorical columns

In [ ]:
for col in cat_cols:
    plt.figure(figsize=(12, 6))
    sns.countplot(data=prev_appdf, x=col, palette='rocket')
    plt.title(f'Count of Each Category in {col}')
    plt.xticks(rotation=45,ha='right')
    plt.show()

#### Bivariate Analysis on Categorical Columns

In [ ]:
for i in range(len(cat_cols)):
    for j in range(i + 1, len(cat_cols)):
        cat_col1 = cat_cols[i]
        cat_col2 = cat_cols[j]
        
        # Create a pivot table to aggregate counts
        pivot_table = pd.crosstab(prev_appdf[cat_col1], prev_appdf[cat_col2])
        
        # Plot heatmap
        plt.figure(figsize=(12, 8))
        sns.heatmap(pivot_table, cmap='rocket', annot=True, fmt='d', linewidths=0.5)
        plt.title(f'Heatmap of {cat_col1} vs. {cat_col2}')
        plt.xlabel(cat_col2)
        plt.ylabel(cat_col1)
        plt.show()

In [ ]:
for cat_col in cat_cols:
    for num_col in num_cols:
        pivot_table = prev_appdf.pivot_table(index=cat_col, values=num_col, aggfunc='mean')
        
        
        plt.figure(figsize=(12, 6))
        sns.heatmap(pivot_table, cmap='rocket', annot=True, fmt='.2f', linewidths=0.5)
        plt.title(f'Heatmap of Mean {num_col} by {cat_col}')
        plt.xlabel(cat_col)
        plt.ylabel('Mean Value')
        plt.show()

In [ ]:
app_df.columns

In [ ]:
prev_appdf.columns

In [ ]:
merged_df = pd.merge(app_df, prev_appdf, on='SK_ID_CURR', how='inner', suffixes=('_curr', '_prev'))

In [ ]:
merged_df = merged_df.drop(columns=['PRODUCT_COMBINATION','Employment_Category'])

In [ ]:
merged_df = merged_df[~merged_df.AMT_CREDIT_prev.isnull()]

In [ ]:
merged_df.columns

In [ ]:
keep_columns = [
    'SK_ID_CURR', 'TARGET', 'SK_ID_PREV',
    'NAME_CONTRACT_TYPE_curr', 'AMT_CREDIT_curr', 'AMT_ANNUITY_curr', 'AMT_GOODS_PRICE_curr',
    'WEEKDAY_APPR_PROCESS_START_curr', 'HOUR_APPR_PROCESS_START_curr',
    'NAME_CONTRACT_TYPE_prev', 'AMT_ANNUITY_prev', 'AMT_CREDIT_prev', 'AMT_GOODS_PRICE_prev',
    'WEEKDAY_APPR_PROCESS_START_prev', 'HOUR_APPR_PROCESS_START_prev', 'AMT_APPLICATION',
    'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
    'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
    'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
    'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
    'ACCOMPANIED_STATUS', 'Income_Category', 'Credit_Category', 'Annuity_Category', 'Goods_Price_Category',
    'AGE_YEARS', 'Age_Category', 'EMPLOYMENT_YEARS', 'REGISTRATION_YEARS', 'Registration_Category',
    'ID_PUBLISH_YEARS', 'ID_Publish_Category',
    'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
    'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'CHANNEL_TYPE',
    'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
    'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
    'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
    'Total_Req_Credit_Enq_Year'
]


In [ ]:
appdata = merged_df[keep_columns]

In [ ]:
appdata.shape

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=appdata, x='NAME_CONTRACT_TYPE_prev', hue='TARGET', palette='rocket')
plt.title('Distribution of Contract Types by Default Status')
plt.xlabel('Contract Type')
plt.ylabel('Count')
plt.legend(title='Default Status', labels=['Non-Defaulter (0)', 'Defaulter (1)'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
numerical_columns_prev = [
    'AMT_APPLICATION',
    'AMT_CREDIT_prev',
    'AMT_ANNUITY_prev',
    'AMT_GOODS_PRICE_prev'
]


for col in numerical_columns_prev:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=appdata, x=col, hue='TARGET', palette='rocket', bins=30, kde=True)
    plt.title(f'Distribution of {col} by TARGET')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
for col in numerical_columns_prev:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='TARGET', y=col, data=appdata, palette='rocket')
    plt.title(f'{col} by Default Status')
    plt.xlabel('TARGET')
    plt.ylabel(col)
    plt.show()

In [ ]:
appdata['AMT_DIFF_prev'] = appdata['AMT_APPLICATION'] - appdata['AMT_CREDIT_prev']
appdata['AMT_CREDIT_TO_INCOME_prev'] = appdata['AMT_CREDIT_prev'] / appdata['AMT_INCOME_TOTAL']
appdata['ANNUITY_TO_CREDIT_prev'] = appdata['AMT_ANNUITY_prev'] / appdata['AMT_CREDIT_prev']

In [ ]:
derived_features = ['AMT_DIFF_prev', 'AMT_CREDIT_TO_INCOME_prev', 'ANNUITY_TO_CREDIT_prev']

for col in derived_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=appdata, x=col, hue='TARGET', palette='rocket', bins=30, kde=True)
    plt.title(f'Distribution of {col} by TARGET')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
for col in derived_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='TARGET', y=col, data=appdata, palette='rocket')
    plt.title(f'{col} by Default Status')
    plt.xlabel('TARGET')
    plt.ylabel(col)
    plt.show()

In [ ]:
crosstab = pd.crosstab(appdata['NAME_CASH_LOAN_PURPOSE'], appdata['Income_Category'])
print(crosstab)

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='NAME_CASH_LOAN_PURPOSE', hue='TARGET', palette='rocket')
plt.title('NAME_CASH_LOAN_PURPOSE vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('NAME_CASH_LOAN_PURPOSE')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='NAME_INCOME_TYPE', hue='TARGET', palette='rocket')
plt.title('NAME_INCOME_TYPE vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('NAME_INCOME_TYPE')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
appdata.columns

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='NAME_CLIENT_TYPE', hue='TARGET', palette='rocket')
plt.title('NAME_CLIENT_TYPE vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('NAME_CLIENT_TYPE')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='NAME_FAMILY_STATUS', hue='TARGET', palette='rocket')
plt.title('NAME_FAMILY_STATUS vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('NAME_FAMILY_STATUS')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='CNT_CHILDREN', hue='TARGET', palette='rocket')
plt.title('CNT_CHILDREN vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('CNT_CHILDREN')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(data=appdata, x='NAME_CONTRACT_STATUS', hue='TARGET', palette='rocket')
plt.title('NAME_CONTRACT_STATUS vs TARGET')
plt.xticks(rotation=45, ha='right')
plt.xlabel('NAME_CONTRACT_STATUS')
plt.ylabel('Count')
plt.legend(title='TARGET')
plt.show()

In [ ]:
numerical_cols = ['TARGET', 'AMT_ANNUITY_curr', 'AMT_APPLICATION', 'AMT_CREDIT_curr',
    'AMT_GOODS_PRICE_curr', 'AMT_ANNUITY_prev', 'AMT_CREDIT_prev',
    'AMT_GOODS_PRICE_prev', 'AMT_DIFF_prev', 'AMT_CREDIT_TO_INCOME_prev',
    'ANNUITY_TO_CREDIT_prev', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'DAYS_DECISION',
    'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_YEAR', 'Total_Req_Credit_Enq_Year', 'EXT_SOURCE_2',
    'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']

numerical_data = appdata[numerical_cols]

correlation_matrix = numerical_data.corr()

pivot_corr_matrix = correlation_matrix[['TARGET']]

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_corr_matrix, annot=True, cmap='rocket', center=0, fmt=".2f")
plt.title('Correlation of Numerical Columns with TARGET')
plt.show()

In [ ]:
res=pd.pivot_table(data=appdata, index="NAME_CONTRACT_STATUS",columns='NAME_CLIENT_TYPE',values='TARGET')
plt.figure(figsize=(15,8))
sns.heatmap(res, annot=True,cmap='rocket', fmt="g")
plt.show()

### Insights are Presented In PPT

--- END ---